In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
athletes = pd.read_csv('athlete_events.csv')
athletes.shape

(271116, 15)

In [5]:
Population = pd.read_csv('WorldPopulation.csv')
Population.shape

(217, 62)

In [6]:
Population.sample(3)

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
96,Jamaica,JAM,"Population, total",SP.POP.TOTL,1628252.0,1650806.0,1676250.0,1703395.0,1730486.0,1756266.0,...,2790122.0,2804082.0,2817210.0,2829493.0,2840992.0,2851807.0,2862087.0,2871934.0,2881355,NaN
122,Moldova,MDA,"Population, total",SP.POP.TOTL,2544000.0,2605000.0,2664000.0,2721000.0,2774000.0,2825000.0,...,3570108.0,3565604.0,3562045.0,3559986.0,3559519.0,3558566.0,3556397.0,3554108.0,3552000,NaN
187,Chad,TCD,"Population, total",SP.POP.TOTL,3001593.0,3060355.0,3121216.0,3183551.0,3246505.0,3309573.0,...,11133861.0,11502786.0,11887202.0,12288651.0,12705135.0,13133589.0,13569438.0,14009413.0,14452543,NaN


In [7]:
athletes.sample(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
29014,15006,Graeme Thomas Brewer,M,21.0,192.0,96.0,Australia,AUS,1980 Summer,1980,Summer,Moskva,Swimming,Swimming Men's 200 metres Freestyle,Bronze
23724,12467,Iulia Bobeic-Bulie,F,25.0,NaN,NaN,Romania,ROU,1992 Summer,1992,Summer,Barcelona,Rowing,Rowing Women's Coxless Fours,NaN
87344,44288,Birgir Gulaugsson,M,22.0,NaN,NaN,Iceland,ISL,1964 Winter,1964,Winter,Innsbruck,Cross Country Skiing,Cross Country Skiing Men's 30 kilometres,NaN


In [33]:
city = athletes['City'] == 'Beijing'
athletes[city].head(3)
athletes[city].shape

(13602, 15)

In [56]:
population_2008 = Population[['Country Code', 'Indicator Name', '2008']]

In [62]:
a = athletes.set_index('NOC').join(population_2008.set_index('Country Code'), how='inner')

In [63]:
a.shape

(197470, 16)

In [64]:
a

,ID,Name,Sex,Age,Height,Weight,Team,Games,Year,Season,City,Sport,Event,Medal,Indicator Name,2008
AFG,502,Ahmad Shah Abouwi,M,NaN,NaN,NaN,Afghanistan,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey Men's Hockey,NaN,"Population, total",27294031.0
AFG,1076,Jammal-ud-Din Affendi,M,28.0,NaN,NaN,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,"Population, total",27294031.0
AFG,1101,Mohammad Anwar Afzal,M,NaN,NaN,NaN,Afghanistan,1948 Summer,1948,Summer,London,Football,Football Men's Football,NaN,"Population, total",27294031.0
AFG,1745,Mohammad Aktar,M,17.0,156.0,48.0,Afghanistan,1980 Summer,1980,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",NaN,"Population, total",27294031.0
AFG,4628,Mohammad Daoud Anwary,M,22.0,NaN,NaN,Afghanistan,1964 Summer,1964,Summer,Tokyo,Wrestling,"Wrestling Men's Bantamweight, Freestyle",NaN,"Population, total",27294031.0
AFG,5285,Mohammad Arref,M,NaN,NaN,52.0,Afghanistan,1972 Summer,1972,Summer,Munich,Wrestling,"Wrestling Men's Flyweight, Freestyle",NaN,"Population, total",27294031.0
AFG,5582,Mohammad Asif,M,17.0,NaN,NaN,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,"Population, total",27294031.0
AFG,5678,Abdul Ghafoor Assar,M,NaN,NaN,NaN,Afghanistan,1948 Summer,1948,Summer,London,Football,Football Men's Football,NaN,"Population, total",27294031.0
AFG,5679,Abdul Ghani Assar,M,NaN,NaN,NaN,Afghanistan,1948 Summer,1948,Summer,London,Football,Football Men's Football,NaN,"Population, total",27294031.0
AFG,5841,Sayed Ali Atta,M,22.0,NaN,NaN,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,"Population, total",27294031.0
